# Eligiendo los mejores hiperparámetros para el Ensemble con GridSearchCV

Ahora vamos a introducir una mejora al proceso anterior, es posible utilizar GridSearchCV para obtener los mejores valores para los hiperparámetros de los modelos que conforman el Ensemble

### GridSearchCV con VotingClassifier - Implementación en Scikit-Learn

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score

#### Los Datos

In [ ]:
df=pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv', sep=';')
df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [ ]:
X=df.drop(axis=1,columns='quality')
y=df['quality']

Por las dudas vamos a desordenar un poco lo datos originales por si fueron cargados de alguna manera en particular

In [ ]:
from sklearn.utils import shuffle
X,y=shuffle(X,y, random_state=123)

In [ ]:
# X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=123)

#### Los Modelos y sus hiperparámetros


Elijamos modelos que sean lo más distintos posible, por ejemplo:

- Regresión Logística con Ridge
- kNN
- Árbol
- SVM

##### Regresión Logística + Ridge

In [ ]:
ridge=linear_model.LogisticRegression(max_iter= 20000,penalty='l2',fit_intercept=True, random_state=123)

##### kNN

In [ ]:
knn=KNeighborsClassifier(n_jobs=-1)

##### Árbol

In [ ]:
tree=DecisionTreeClassifier(random_state=123)

##### SVM

In [ ]:
svm=SVC( random_state=123)

#### Creamos el "meta modelo" con el ensemble del VotingClassifier

In [ ]:
from sklearn.ensemble import VotingClassifier

In [ ]:
modelos=[('ridge',ridge),('knn',knn),('arbol',tree),('svm',svm)]

ensemble=VotingClassifier(estimators=modelos,voting='hard',n_jobs=-1) #, weights=[2,1,3,1])

### GridSearchCV

Para utilizar GridSearchCV tenemos que pasarle al modelo los valores que deseamos que pruebe para sus hiperparámetros.   

En nuestro caso dichos hiperparámetros serán los que corresponden a cada uno de los modelos que forman parte del Ensemble.

#### Hiperparámetros

Crearemos un diccionario con los valores del los hiperparámetros de cada uno de los modelos

In [ ]:
hiperparametros={}

Para ridge el hiperparámetro que solemos utilizar es el coeficiente de regularización denominado 'C', lo agregramos al diccionario hiperparametros con las lista de valores que queremos que pruebe.  

**ATENCIÓN:** Cuando se pasa el nombre del estimador definido anteriormente, 'ridge' por ejemplo, **luego hay que escribir doble guión bajo __ seguido del nombre del (hiper)parámetro**  

**Nota:** Para que no demore tanto, pasaremos sólo algunos valores de hiperparámetros.

Podemos verificar la lista de hiperparametros con el siguiente comando

In [ ]:
ensemble.get_params().keys()

dict_keys(['estimators', 'flatten_transform', 'n_jobs', 'verbose', 'voting', 'weights', 'ridge', 'knn', 'arbol', 'svm', 'ridge__C', 'ridge__class_weight', 'ridge__dual', 'ridge__fit_intercept', 'ridge__intercept_scaling', 'ridge__l1_ratio', 'ridge__max_iter', 'ridge__multi_class', 'ridge__n_jobs', 'ridge__penalty', 'ridge__random_state', 'ridge__solver', 'ridge__tol', 'ridge__verbose', 'ridge__warm_start', 'knn__algorithm', 'knn__leaf_size', 'knn__metric', 'knn__metric_params', 'knn__n_jobs', 'knn__n_neighbors', 'knn__p', 'knn__weights', 'arbol__ccp_alpha', 'arbol__class_weight', 'arbol__criterion', 'arbol__max_depth', 'arbol__max_features', 'arbol__max_leaf_nodes', 'arbol__min_impurity_decrease', 'arbol__min_samples_leaf', 'arbol__min_samples_split', 'arbol__min_weight_fraction_leaf', 'arbol__random_state', 'arbol__splitter', 'svm__C', 'svm__break_ties', 'svm__cache_size', 'svm__class_weight', 'svm__coef0', 'svm__decision_function_shape', 'svm__degree', 'svm__gamma', 'svm__kernel', 's

Entonces los hiperparametros para el modelo de ridge sería

In [ ]:
hiperparametros['ridge__C']=[1e-1,1,1e1] # [1e-4,1e-3,1e-2,1e-1,1,1e1,1e2,1e3]

Para KNN el hiperparámetro que solemos utilizar es la cantidad de vecinos denominado  'n_neighbors'

In [ ]:
hiperparametros['knn__n_neighbors']=[5,7]  #[1,2,3,4,5,6,7,8,9,10,11,12,13]

Para árboles de decisiones el hiperparámetro que solemos utilizar es la profundidad del mismo, denominado 'max_depth'

In [ ]:
hiperparametros['arbol__max_depth']=[1,3,7,9]  # [1,2,3,4,5,6,7,8,9,10,11,12,13]

Para Support Vector Machine el hiperparámetro utilizado también es el coeficiente de Regularización 'C'

In [ ]:
hiperparametros['svm__C']=[1e-3, 1e-2,1e3] # [1e-4,1e-3,1e-2,1e-1,1,1e1,1e2,1e3]

Finalmente veamos todos los hiperparámetros en un unico diccionario

In [ ]:
hiperparametros

{'ridge__C': [0.1, 1, 10.0],
 'knn__n_neighbors': [5, 7],
 'arbol__max_depth': [1, 3, 7, 9],
 'svm__C': [0.001, 0.01, 13]}

#### Creamos el GridSearchCV

Creamos un GridSearchCV pasando como modelo o estimator el **ensemble** y como param_grid nuestro diccionario de hiperparámetros

In [ ]:
from sklearn.model_selection import GridSearchCV

grid_ensemble= GridSearchCV(estimator=ensemble, param_grid=hiperparametros, cv=5,n_jobs=-1, return_train_score=True)


##### Lo entrenamos con fit 

In [ ]:
# esto va a demorar ...
grid_ensemble.fit(X, y);

In [ ]:
grid_ensemble.best_score_

0.5569610806528945

Curiosamente nos está dando peor que antes, sin hacer gridsearchcv.

In [ ]:
grid_ensemble.best_params_

{'arbol__max_depth': 9, 'knn__n_neighbors': 5, 'ridge__C': 1, 'svm__C': 0.001}